### Классификация текстов: спам-фильтр для SMS

In [2]:
import pandas as pd

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

#### Задание

1. Загрузите датасет. Описание датасета можно посмотреть здесь.

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [17]:
data = pd.read_csv('SMSSpamCollection.txt', delimiter = "\t", header=None, names=['label', 'text'])

In [18]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [19]:
text=data.text
y=data['label'].apply(lambda x:1 if x=='spam' else 0)

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

In [22]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [31]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [32]:
cv = StratifiedKFold(n_splits=10, random_state=2) 
model_1 = cross_val_score(LogisticRegression(), X, y, cv=cv, scoring='f1')

C:\Users\Rollan\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [36]:
round(model_1.mean(),1)

0.9

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [43]:
test_text = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", 
             "FreeMsg: Txt: claim your reward of 3 hours talk time",
             "Have you visited the last lecture on physics?",
             "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
             "Only 99$"]
X_test=vectorizer.transform(test_text)

LogReg = LogisticRegression()
LogReg.fit(X,y)
Pred = LogReg.predict(X_test)
print(Pred)

[1 1 0 0 0]


7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [46]:
vectorizer = CountVectorizer(ngram_range=(2,2))
X = vectorizer.fit_transform(text)
model_2 = cross_val_score(LogisticRegression(), X, y, cv=cv, scoring='f1')
round(model_2.mean(),2)

C:\Users\Rollan\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.82

In [47]:
vectorizer = CountVectorizer(ngram_range=(3,3))
X = vectorizer.fit_transform(text)
model_3 = cross_val_score(LogisticRegression(), X, y, cv=cv, scoring='f1')
round(model_3.mean(),2)

C:\Users\Rollan\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.73

In [48]:
vectorizer = CountVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(text)
model_4 = cross_val_score(LogisticRegression(), X, y, cv=cv, scoring='f1')
round(model_4.mean(),2)

C:\Users\Rollan\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.93

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [50]:
from sklearn.naive_bayes import MultinomialNB
vectorizer = CountVectorizer(ngram_range=(2,2))
X = vectorizer.fit_transform(text)
model_5 = cross_val_score(MultinomialNB(), X, y, cv=cv, scoring='f1')
round(model_5.mean(),2)

0.65

In [51]:
vectorizer = CountVectorizer(ngram_range=(3,3))
X = vectorizer.fit_transform(text)
model_6 = cross_val_score(MultinomialNB(), X, y, cv=cv, scoring='f1')
round(model_6.mean(),2)

0.38

In [52]:
vectorizer = CountVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(text)
model_7 = cross_val_score(MultinomialNB(), X, y, cv=cv, scoring='f1')
round(model_7.mean(),2)

0.89

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [58]:
Tfidvec=TfidfVectorizer()
X=Tfidvec.fit_transform(text)
score=cross_val_score(LogisticRegression(),X,y,cv=cv,scoring='f1')
round(score.mean(),2)

C:\Users\Rollan\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.85